# climpy — Demo notebook

Funcționează pe **Google Colab**, **Binder** și orice Jupyter local.

**Pași:**
1. Rulează celula de Setup (instalează pachetele + climpy)
2. Generează datele de test sintetice
3. Preprocessing → EOF → CCA → Corelații
4. Figuri calitate publicație


## 0 — Setup (rulează o singură dată)

> Pe **Colab**: după upload-ul zip-ului, rulează celulele de mai jos în ordine.  
> Pe **Binder**: pachetele sunt deja instalate, sari direct la secțiunea 1.

In [ ]:
# ── Doar pe Google Colab ──────────────────────────────────────────────
# Dezarhivează climpy_v0.2.0.zip (trebuie uploadat mai întâi în Files)

import os, sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Dezarhivare
    !unzip -q climpy_v0.2.0.zip
    
    # Instalare dependențe
    !pip install -q cartopy eofs
    
    # Instalare climpy în modul editabil
    !pip install -q -e climpy_package/
    
    print('✓ climpy instalat pe Colab')
else:
    # Pe Binder sau Jupyter local — climpy e deja instalat
    print('✓ Mediu non-Colab detectat — skip setup')

In [ ]:
import climpy
print(f'climpy {climpy.__version__} — OK')

## 1 — Generare date de test sintetice

In [ ]:
# Generează 3 fișiere NetCDF mici (~3 MB total) cu date climatice sintetice
%run climpy_package/generate_test_data.py

## 2 — Preprocessing

In [ ]:
from pathlib import Path

DATA = Path('data/test')

# Aplică tot preprocessing-ul dintr-un apel:
# anomalii, medii anuale, medii sezoniere DJFM, scade media globală
sst_result = climpy.preprocess(
    DATA / 'sst_test.nc',
    var            = 'Xdata',
    lon_convention = '[-180,180]',
    ref_period     = ('1981-01', '2010-12'),
    season_months  = [12, 1, 2, 3],    # DJFM
    subtract_gm    = True,
)

anom   = sst_result['anom']
annual = sst_result['annual']
djfm   = sst_result['seasonal_no_gm']

print('Anomalii lunare:', dict(anom.sizes))
print('Medii anuale:   ', dict(annual.sizes))
print('DJFM (- GW):    ', dict(djfm.sizes))

## 3 — Inspecție vizuală rapidă

In [ ]:
import matplotlib.pyplot as plt
climpy.use_style('nature')

fig = climpy.ClimPlot(
    nrows=1, ncols=3, w=climpy.NATURE_2COL, h=3,
    map_proj=(climpy.Map(), climpy.Map(), climpy.Map()),
)

times = anom['time'].values
fig[0].map(anom.isel(time=0),  title=f'{str(times[0])[:7]}',
           cbar_label='°C', diverging=True)
fig[1].map(anom.isel(time=len(times)//2), title=f'{str(times[len(times)//2])[:7]}',
           cbar_label='°C', diverging=True)
fig[2].map(anom.isel(time=-1), title=f'{str(times[-1])[:7]}',
           cbar_label='°C', diverging=True)

fig.label_subplots()
plt.suptitle('SST anomalii — verificare date', y=1.02)
plt.show()

## 4 — Analiză EOF

In [ ]:
solver = climpy.EOF(djfm, min_variance=70)
solver.summary()

In [ ]:
eofs = solver.eofs()
pcs  = solver.pcs()
frac = solver.variance_fraction()

# Construim AMO și Tripol
eof_amo    = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=1, b= 1)
eof_tripol = climpy.lincomb(eofs.isel(mode=0), eofs.isel(mode=1), a=1, b=-1)
pc_amo     = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=1, b= 1)
pc_tripol  = climpy.lincomb(pcs.isel(mode=0),  pcs.isel(mode=1),  a=1, b=-1)

pc_amo_ma    = climpy.moving_average(pc_amo,    n=7)
pc_tripol_ma = climpy.moving_average(pc_tripol, n=7)

print('EOFs shape:', eofs.shape)
print('PCs  shape:', pcs.shape)

In [ ]:
climpy.use_style('nature')

fig = climpy.ClimPlot(
    nrows=2, ncols=2,
    w=climpy.NATURE_2COL, h=5.5,
    map_proj=(
        climpy.Globe(central_longitude=-30, central_latitude=45),
        climpy.Globe(central_longitude=-30, central_latitude=45),
        'ts', 'ts',
    ),
)

fig[0].map(eof_amo,
           title=f'EOF-AMO ({frac.values[0]:.1f} %)',
           filltype='contourf', vmin=-0.6, vmax=0.1,
           cbar_step=0.1, cbar_label='SST anomaly (°C)',
           cmap=plt.cm.RdBu_r)

fig[1].map(eof_tripol,
           title=f'EOF-Tripol ({frac.values[1]:.1f} %)',
           filltype='contourf', vmin=-0.6, vmax=0.6,
           cbar_step=0.2, cbar_label='SST anomaly (°C)',
           cmap=plt.cm.RdBu_r)

fig[2].ts(pc_amo, pc_amo_ma,
          title='PC-AMO',
          labels=['AMO', '7-yr MA'],
          colors=['steelblue', 'firebrick'],
          alphas=[0.35, 1.0], linewidths=[0.8, 1.8])

fig[3].ts(pc_tripol, pc_tripol_ma,
          title='PC-Tripole',
          labels=['Tripole', '7-yr MA'],
          colors=['steelblue', 'firebrick'],
          alphas=[0.35, 1.0], linewidths=[0.8, 1.8])

fig.label_subplots()
fig.savefig('figures/fig_eof_test.pdf')
plt.show()

## 5 — CCA: SST ↔ SLP

In [ ]:
# Preprocessing SLP
slp_result = climpy.preprocess(
    DATA / 'slp_test.nc',
    var           = 'Xdata',
    ref_period    = ('1981-01', '2010-12'),
    season_months = [12, 1, 2, 3],
)
slp_djfm = slp_result['seasonal']

# CCA
cca = climpy.CCA(djfm, slp_djfm, min_variance=70)
cca.summary()

In [ ]:
Xpat  = cca.x_patterns()
Ypat  = cca.y_patterns()
Xts   = cca.x_timeseries()
Yts   = cca.y_timeseries()
corrs = cca.canonical_correlations()
varX, varY = cca.variance_fraction()

fig = climpy.ClimPlot(
    nrows=3, ncols=2, w=climpy.NATURE_2COL, h=8,
    map_proj=(
        climpy.Map(), climpy.Map(),
        climpy.Map(), climpy.Map(),
        'ts',         'ts',
    ),
)

for col, mode in enumerate([0, 1]):
    m = mode + 1
    fig[col].map(Xpat.isel(mode=mode),
                 title=f'SST CCP-{m} ({varX[mode]:.1f} %)',
                 lon_ticks=[-75,-45,-15,15],
                 lat_ticks=[0,20,40,60,80],
                 map_extent=[-80,20,0,80],
                 cbar_label='°C', cmap=plt.cm.RdBu_r)

    fig[2 + col].map(Ypat.isel(mode=mode),
                     title=f'SLP CCP-{m} ({varY[mode]:.1f} %)',
                     lon_ticks=[-180,-90,0,90,180],
                     lat_ticks=[-60,-30,0,30,60,90],
                     cbar_label='hPa', cmap=plt.cm.RdBu_r)

    fig[4 + col].ts(Xts.isel(mode=mode), Yts.isel(mode=mode),
                    title=f'Pair {m}  (r = {corrs[mode]:.2f})',
                    labels=['SST', 'SLP'],
                    colors=['steelblue','firebrick'])

fig.label_subplots()
fig.savefig('figures/fig_cca_test.pdf')
plt.show()

## 6 — Corelații cu stippling

In [ ]:
# Preprocessing precipitații
pr_result = climpy.preprocess(
    DATA / 'precip_test.nc',
    var           = 'Xdata',
    ref_period    = ('1981-01', '2010-12'),
    season_months = [12, 1, 2, 3],
)
pr_djfm = pr_result['seasonal'] * 86400   # → mm/day

# Corelație + p-values (pentru stippling)
r_amo,    p_amo    = climpy.correlation_pvalue(pc_amo,    pr_djfm, dim='year')
r_tripol, p_tripol = climpy.correlation_pvalue(pc_tripol, pr_djfm, dim='year')

print(f'Max |r| AMO:    {abs(r_amo).max().item():.3f}')
print(f'Max |r| Tripol: {abs(r_tripol).max().item():.3f}')

In [ ]:
fig = climpy.ClimPlot(
    nrows=1, ncols=2, w=climpy.NATURE_2COL, h=3.5,
    map_proj=(climpy.Map(-65), climpy.Map(-65)),
)

kw = dict(filltype='contourf', cmap=plt.cm.BrBG,
          vmin=-0.6, vmax=0.6, cbar_orientation='horizontal',
          cbar_label='r')

fig[0].map(r_amo,    title='AMO × Precip. DJFM', **kw)
fig.add_stippling(fig.axes[0], p_amo.values,
                  r_amo['lat'].values, r_amo['lon'].values, pthresh=0.05)

fig[1].map(r_tripol, title='Tripole × Precip. DJFM', **kw)
fig.add_stippling(fig.axes[1], p_tripol.values,
                  r_tripol['lat'].values, r_tripol['lon'].values, pthresh=0.05)

fig.label_subplots()
fig.savefig('figures/fig_corr_test.pdf')
plt.show()
print('Punctele negre = semnificativ (p < 0.05)')

---
## 7 — Lag correlations

In [ ]:
# Efect al AMO cu 10 ani în viitor
pc_lag, pr_lag = climpy.lag_pair(pc_amo, pr_djfm, k=10, dim='year')
r_lag10, p_lag10 = climpy.correlation_pvalue(pc_lag, pr_lag, dim='year')

fig = climpy.ClimPlot(
    nrows=1, ncols=1, w=climpy.NATURE_15COL, h=3.5,
    map_proj=(climpy.Map(-65),),
)

fig[0].map(r_lag10,
           title='AMO (t) × Precip. DJFM (t+10 ani)',
           filltype='contourf', cmap=plt.cm.BrBG,
           vmin=-0.6, vmax=0.6,
           cbar_orientation='horizontal', cbar_label='r')
fig.add_stippling(fig.axes[0], p_lag10.values,
                  r_lag10['lat'].values, r_lag10['lon'].values)

fig.label_subplots()
fig.savefig('figures/fig_lagcorr_test.pdf')
plt.show()

---
Figurile salvate în `figures/` sunt PDF vectoriale, gata de inclus în articol.